
## Automatic Differentiation with ``torch.autograd``

When training neural networks, the most frequently used algorithm is **back propagation**.   

In this algorithm, parameters (model weights) are adjusted   

according to the **gradient** of the loss function with respect to the given parameter.  
adjusted : 조절됨 - 손실 함수의 기울기에 따라  

To compute those gradients, PyTorch has a built-in differentiation engine called ``torch.autograd``.   
자동미분    
It supports automatic computation of gradient for any computational graph.  

Consider the simplest one-layer neural network, with input ``x``,
parameters ``w`` and ``b``, and some loss function.   
It can be defined in PyTorch in the following manner:

In [8]:
import torch

x = torch.ones(5)  # input tensor
# 크기가 5인 1로 채워진 텐서

y = torch.zeros(3)  # expected output
# 크기가 3인 0으로 채워진 텐서

w = torch.randn(5, 3, requires_grad=True)
# 크기가 (5, 3)인 무작위로 초기화된 텐서
# 모델의 weight
# requires_grad=True : 이 텐서에 대한 그래디언트 계산이 필요하다는 것을 나타냄

b = torch.randn(3, requires_grad=True)
# 크기가 3인 무작위로 초기화된 텐서 
# 모델의 bias

z = torch.matmul(x, w)+b
# torch.matmul(x, w): 입력 텐서 x와, weight 텐서 w 간의 행렬 곱셈을 수행
# x는 1차원 텐서이고, w는 2차원 텐서이므로, 내적(inner product)과 동일
# 결과로 나오는 텐서는 1차원 텐서
# + b 부분은 행렬 곱셈의 결과에 편향 b를 더함
# 편향은 각각의 출력에 더해져서 최종적인 출력 z를 얻게 됨

loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
# logit 값 z와 실제 타겟 y 사이의 이진 교차 엔트로피 손실을 계산

### Tensors, Functions and Computational graph
  
This code defines the following computational graph:  


/_static/img/basics/comp-graph.png

In this network, w and b are **parameters**, which we need to optimize.   

Thus, we need to be able to compute the gradients of loss function with respect to those variables.   
In order to do that, we set the ``requires_grad`` property of those tensors.

PyTorch의 autograd 패키지는 계산 그래프(computation graph)를 활용하여 텐서 연산에 대한 그래디언트를 자동으로 계산  

requires_grad=True: 텐서를 생성할 때 이 속성을 True로 설정하면,   
해당 텐서의 모든 연산이 추적되며, 계산 그래프에 해당 연산이 기록됨   

이 텐서의 .grad 속성을 통해 그래디언트를 얻을 수 있음  
이는 주로 모델의 가중치나 학습 가능한 매개변수에 적용됨  

requires_grad=False (기본값): 텐서를 생성할 때 이 속성을 False로 설정하면, 해당 텐서의 연산은 추적되지 않음  
그래디언트를 계산할 필요가 없는 경우에 사용됨   
이 설정을 통해 연산의 속도를 향상시키고 메모리를 절약할 수 있음

#### NOTE
You can set the value of ``requires_grad`` when creating a tensor,   
or later by using ``x.requires_grad_(True)`` method.

A function that we apply to tensors to construct computational graph is in fact an object of class Function.   

This object knows how to compute the function in the forward direction,   

and also how to compute its derivative(도함수) during the backward propagation step. 

A reference to the backward propagation function is stored in grad_fn property of a tensor.   
텐서의 grad_fn 속성에 역전파 함수에 대한 참조가 저장  

You can find more information of Function in the documentation.  

In [9]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x000001C55A278970>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x000001C55A2004C0>


### Computing Gradients

To optimize weights of parameters in the neural network,   

we need to compute the derivatives(도함수) of our loss function with respect to parameters,  

namely, we need $\frac{\partial loss}{\partial w}$ and
$\frac{\partial loss}{\partial b}$ under some fixed values of
``x`` and ``y``.   

$\frac{\partial loss}{\partial w}$ : 손실 함수를 가중치 w에 대해 편미분  
일반적으로 딥러닝에서는 손실 함수를 최소화하는 것이 목표이므로,  
가중치 w를 조금 변경했을 때 손실 함수가 얼마나 변화하는지 나타냄  

To compute those derivatives, we call
``loss.backward()``,   

자동 미분(autograd)을 사용하여 그래디언트(기울기)를 계산하고 각 매개변수의 .grad 속성에 저장  

and then retrieve the values from ``w.grad`` and
``b.grad``:  

retrieve : 정보, 데이터, 값 등을 가져오거나 얻어오는 것

In [10]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2811, 0.2018, 0.1770],
        [0.2811, 0.2018, 0.1770],
        [0.2811, 0.2018, 0.1770],
        [0.2811, 0.2018, 0.1770],
        [0.2811, 0.2018, 0.1770]])
tensor([0.2811, 0.2018, 0.1770])


#### NOTE

- We can only obtain(구하다) the grad properties for the leaf nodes of the computational graph,   
which have ``requires_grad`` property set to ``True``.   
For all other nodes in our graph, gradients will not be available.  

- We can only perform gradient calculations using backward once on a given graph, for performance reasons.   
주어진 그래프에서는 성능상의 이유로 한 번만 역전파를 수행할 수 있음  
(한 번의 역전파 후에는 이 그래프가 해제되고 메모리에서 삭제됨, 이렇게 함으로써 성능을 향상)  
If we need to do several backward calls on the same graph,  
동일한 그래프에 대해 여러 번 역전파를 수행해야 하는 경우  
we need to pass ``retain_graph=True`` to the backward call.  
retain_graph=True를 backward 호출에 전달하여 그래프를 유지하도록 할 수 있음  

### Disabling Gradient Tracking
By default, all tensors with ``requires_grad=True`` are tracking their computational history and support gradient computation.   
기본적으로 requires_grad=True로 설정된 모든 텐서는 계산 이력을 추적하며 그래디언트 계산을 지원  

However, there are some cases when we do not need to do that,   

for example, when we have trained the model and just want to apply it to some input data,   
모델을 훈련시키고 나서 입력 데이터에 모델을 적용하기만 하려는 경우  

i.e. we only want to do forward computations through the network.   
 네트워크를 통해 전방 계산만 수행하고 그래디언트 계산이 필요하지 않을 때  
 
We can stop tracking computations by surrounding our computation code with ``torch.no_grad()`` block:  
torch.no_grad() 블록으로 계산 코드를 감싸면 됨

In [11]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


Another way to achieve the same result is to use the ``detach()`` method on the tensor:


In [12]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


There are reasons you might want to disable gradient tracking:
  - To mark some parameters in your neural network as **frozen parameters**.

  신경망의 일부 매개변수를 frozen parameters로 표시  
  모델을 훈련할 때 특정 매개변수의 업데이트를 막아야 하는 경우가 있음  
  이를 통해 미리 학습된 가중치를 고정하고 새로운 데이터에 대한 전방 전파만 수행할 수 있음  
  
  - To **speed up computations** when you are only doing forward pass,   
  because computations on tensors that do not track gradients would be more efficient.

  전방 전파만 수행할 때 연산 속도를 높이기 위해:   
  그래디언트를 추적하지 않는 텐서의 연산은 더 효율적으로 수행됨  
  따라서 학습 중이 아닌 단계에서 속도를 높이고 메모리를 절약하기 위해 그래디언트 추적을 비활성화할 수 있음


### More on Computational Graphs
Conceptually(개념적으로), autograd keeps a record of data (tensors)   
and all executed operations (along with the resulting new tensors)   
in a directed acyclic graph (DAG) consisting of Function objects.   

autograd는 / Function 객체로 이루어진 DAG에서 / 데이터(텐서)와 / 수행된 모든 연산을 기록함
  
In this DAG, leaves are the input tensors, roots are the output tensors.   
By tracing this graph from roots to leaves,  
you can automatically compute the gradients using the chain rule.  

DAG에서 잎(Leaves)은 입력 텐서이고, 뿌리(Roots)는 출력 텐서  
이 그래프를 뿌리에서 잎까지 추적함으로써 연쇄 법칙(chain rule)을 사용하여 자동으로 그래디언트를 계산할 수 있음  

In a forward pass, autograd does two things simultaneously(동시에):

- run the requested operation to compute a resulting tensor  
요청된 연산을 실행하여 결과 텐서를 계산
- maintain the operation’s *gradient function* in the DAG.  
DAG에서 연산의 그래디언트 함수를 유지

The backward pass kicks off when ``.backward()`` is called on the DAG
root. ``autograd`` then:  
역전파는 DAG의 루트에서 .backward()가 호출될 때 시작됨, 그 후 autograd는 :  

- computes the gradients from each ``.grad_fn``,
- accumulates(누적) them in the respective(각각의) tensor’s ``.grad`` attribute
- using the chain rule, propagates all the way to the leaf tensors.  
연쇄 법칙을 사용하여 leaf tensors까지 전파

##### NOTE
**DAGs are dynamic in PyTorch**   
각각의 역전파 단계에서 새로운 그래프가 생성되어 동적인 모델 구성이 가능하게 됨

An important thing to note is that the graph is recreated from scratch;   
그래프가 매번 새로 생성  
from scratch : 아무 것도 없는 상태에서 시작한다는 의미  

after each .backward() call, autograd starts populating a new graph.   
각 .backward() 호출 이후에 autograd는 새로운 그래프를 구성  

This is exactly what allows you to use control flow statements in your model;   
모델에서 제어 흐름 문장을 사용할 수 있는 것을 정확히 의미  

you can change the shape, size and operations at every iteration if needed.  
필요한 경우 각 반복에서 모양, 크기 및 연산을 변경할 수 있음

### Optional Reading: Tensor Gradients and Jacobian Products

In many cases, we have a scalar loss function,   
and we need to compute the gradient with respect to some parameters.  
 with respect to : ~에 관하여
  
However, there are cases
when the output function is an arbitrary(임의의) tensor.   
In this case, PyTorch allows you to compute so-called **Jacobian product**,  
and not the actual gradient.

For a vector function $\vec{y}=f(\vec{x})$, where
$\vec{x}=\langle x_1,\dots,x_n\rangle$ and
$\vec{y}=\langle y_1,\dots,y_m\rangle$,   

벡터 함수 $\vec{y}=f(\vec{x})$가 있을 때,
$\vec{x}=\langle x_1,\dots,x_n\rangle$이고
$\vec{y}=\langle y_1,\dots,y_m\rangle$이라면,  


a gradient of $\vec{y}$ with respect to $\vec{x}$ is given by **Jacobian
matrix**:

$\vec{y}$에 대한 $\vec{x}$의 그래디언트  

\begin{align}J=\left(\begin{array}{ccc}
      \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
      \vdots & \ddots & \vdots\\
      \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
      \end{array}\right)\end{align}

<br>
<br>
Instead of computing the Jacobian matrix itself,   
Jacobian 행렬 자체를 계산하는 대신,   

PyTorch allows you to compute **Jacobian Product** $v^T\cdot J$ for a given input vector
$v=(v_1 \dots v_m)$.   
주어진 입력 벡터 $v=(v_1 \dots v_m)$에 대한 Jacobian Product $v^T\cdot J$를 계산할 수 있음  

This is achieved by calling ``backward`` with $v$ as an argument.   
이는 backward를 $v$와 함께 호출함으로써 달성  

The size of $v$ should be the same as the size of the original tensor,   
$v$의 크기는 원래 텐서와 동일해야 하며,  

with respect to which we want to compute the product:  
계산하려는 그래디언트의 원래 텐서에 대한 크기와 일치해야 함


In [13]:
inp = torch.eye(4, 5, requires_grad=True)
# torch.eye(4, 5, requires_grad=True) :  4x5 크기의 단위 행렬 생성
# 대각 원소가 1이고 나머지는 0인 행렬

out = (inp+1).pow(2).t()
# inp에 1을 더한 후 제곱, transpose

out.backward(torch.ones_like(out), retain_graph=True)
# out에 대한 그래디언트 계산
# torch.ones_like(out) : out과 같은 크기의 1로 이루어진 텐서를 생성
# 이를 사용하여 out.backward를 호출하여 그래디언트 계산
# retain_graph=True : 그래프를 유지하도록 지정

print(f"First call\n{inp.grad}")  # 그래디언트 출력

out.backward(torch.ones_like(out), retain_graph=True)
# 그래프를 다시 사용하여 out에 대한 그래디언트 계산

print(f"\nSecond call\n{inp.grad}")
# 다시 그래디언트를 출력, 이전에 계산된 그래디언트에 현재 계산된 그래디언트가 더해지게 됨

inp.grad.zero_() # inp.grad를 0으로 초기화

out.backward(torch.ones_like(out), retain_graph=True)
# 그래프를 다시 사용하여 out에 대한 그래디언트를 계산
# inp.grad는 이전에 초기화되었기 때문에 현재 계산된 그래디언트로 덮어씌워짐

print(f"\nCall after zeroing gradients\n{inp.grad}")
# 마지막으로 그래디언트 출력, 초기화 후에 계산된 그래디언트임

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])


Notice that when we call ``backward`` for the second time with the same
argument,   
the value of the gradient is different.   
동일한 인자로/ 두번째로 backward를 호출할 때 그래디언트 값이 다르다는 점에 주목  

This happens because when doing ``backward`` propagation, PyTorch **accumulates the
gradients**, i.e.   
backward 역전파를 수행할 때 PyTorch가 그래디언트를 **누적(accumulate)**하기 때문  

the value of computed gradients is added to the
``grad`` property of all leaf nodes of computational graph.    
즉, 계산된 그래디언트의 값이 계산 그래프의 모든 leaf nodes의 grad 속성에 추가됨  

If you want to compute the proper gradients, you need to zero out the ``grad`` property before.   
올바른 그래디언트를 계산하려면 backward를 호출하기 전에 grad 속성을 0으로 초기화해야 함  

In real-life training an *optimizer* helps us to do this.


##### NOTE
Previously we were calling backward() function without parameters.   
This is essentially equivalent to calling backward(torch.tensor(1.0)),   

매개변수 없이 backward() 함수 호출은  본질적으로 backward(torch.tensor(1.0))을 호출하는 것과 동등  

which is a useful way to compute the gradients in case of a scalar-valued function,   
손실 함수와 같이 스칼라 값 함수에 대한 그래디언트를 계산하는 유용한 방법    

such as loss during neural network training.  
특히 신경망 훈련 중에 손실을 다룰 때 사용됨